In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# Load Datasets
uc = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/uc_expanded.csv')
cd = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/cd_expanded.csv')
healthy = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/healthy_expanded.csv')

In [58]:
# Make a list of the lab values (column headers)
lab_value_names = list(healthy.columns.values)
lab_value_names = lab_value_names[1:-1]

# Make a list of the DataFrames
all_dfs = [uc, cd, healthy]

In [63]:
# Normalize Each Patient's Data
# Min/Max scaling [0,1] and account for missing values
def minmax_scale(x, min, max):
    ''' 
    Min/Max Scaler [0,1] that also handles when the max=min.
    Inputs:
    x (int) - Value to be normalized
    min (int) - minmum value
    max (int) - maximum value

    Outputs:
    norm (int) - normalized value [0,1]
    '''
    if min != max:
        scaled = max - min
    else:
        scaled = 1
    norm = (x - min) / scaled
    return norm


# Need to calculate a mean normalized value in the event that a given patient only has one recorded value or no values.
# Mean Noramlized Values

# Lists to store patient normalized dataframes
healthy_pts = []
uc_pts = []
cd_pts = []
pts_lsts = [healthy_pts, uc_pts, cd_pts]

for pt_pop, df in enumerate(all_dfs):
    mean_norm_values = []
    for lab in lab_value_names:
        #print("Lab Name: ", lab)
        values = df[lab].fillna(0)
        min_lab = min(values)
        #print("Minimum Value: ", min_lab)
        max_lab = max(values)
        #print("Maximum Value: ", max_lab)
        mean_value = values.mean()
        norm_lab_val = minmax_scale(mean_value, min_lab, max_lab)
        #print('Scaled Value: ', norm_lab_val, '\n')
        mean_norm_values.append(norm_lab_val)

    patient_id_lst = list(df.patient_id.unique())
    for patient in patient_id_lst:
        patient_df = df[df['patient_id'] == patient].reset_index()
        # Patient Normalized Values
        visits = list(range(0,5))
        na_reference_df = patient_df.copy()
        na_reference_df = na_reference_df.isnull()
        norm_df = patient_df.copy()
        for i, lab in enumerate(lab_value_names):
            #print("\n Lab Name: ", lab)
            values = patient_df[lab].fillna(0)
            min_lab = min(values)
            max_lab = max(values)
            #print("Minimum Value: ", min_lab)
            #print("Maximum Value: ", max_lab)
            for v in visits:
                print(v, lab, na_reference_df)
                is_missing = na_reference_df.loc[v, lab]
                #print("Missing???: ", is_missing)
                if is_missing == True:
                    norm_val = mean_norm_values[i]
                    #print("Missing Norm Val: ", norm_val)
                else:
                    value = df.loc[v, lab]
                    norm_val = minmax_scale(value, min_lab, max_lab)
                #print("Lab Value: ", value)
                #print('Scaled Value: ', norm_val)
                norm_df.loc[v, lab] = norm_val
        pts_lsts[pt_pop].append(norm_df)





0 crp       patient_id    crp    fer  btwelve    fol    alb    alp    alt    pro  \
0          False  False  False     True   True   True   True   True   True   
1688       False   True  False    False   True  False  False  False  False   
3376       False   True  False     True   True   True   True   True   True   
5064       False   True  False    False  False  False  False  False  False   
6752       False   True   True     True   True   True   True   True   True   

        crt    uac    cal    vtd    hgb    mcv    hct    wbc    pmn    plt  \
0      True   True   True   True  False  False  False  False  False  False   
1688  False  False  False  False  False  False  False  False  False  False   
3376   True   True   True   True  False  False  False  False  False  False   
5064  False  False  False   True  False  False  False  False  False  False   
6752   True   True   True   True   True   True   True   True   True   True   

       year  
0     False  
1688  False  
3376  False  


KeyError: 1

In [60]:
list(df.patient_id.unique())

pt_test = df[df['patient_id'] == 1253095] 
pt_test.head()


,patient_id,crp,fer,btwelve,fol,alb,alp,alt,pro,crt,uac,cal,vtd,hgb,mcv,hct,wbc,pmn,plt,year
0,1253095,0.0,23.0,294.0,NaN,3.75,65.5,14.5,6.8,0.65,4.35,9.15,NaN,12.72,81.96,38.58,5.26,2.72,221.6,1
5415,1253095,NaN,35.0,243.0,9.6,3.90,70.0,15.0,6.6,0.60,4.30,8.90,NaN,13.20,83.00,39.60,5.10,2.60,274.0,2
10830,1253095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
16245,1253095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
21660,1253095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
